In [96]:
import pandas as pd
import numpy as np
import copy

# Camiones y Camión

In [290]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes")
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [291]:
df_camiones = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="camiones")
df_camiones

,camion,carga_max,clientes_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [ ]:
df_distancias = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="distancias", index_col=[0])
df_distancias

In [215]:
for cod in list(df_clientes.index)[1:]:
    print(round(((df_clientes.loc["A"]["Coord x"] - df_clientes.loc[cod]["Coord x"])**2 + (df_clientes.loc["A"]["Coord y"] - df_clientes.loc[cod]["Coord y"])**2)**(1/2), 1))

1.3
0.9
0.8
1.2
0.8
2.1
2.2
1.5
2.2
2.9
2.0
1.5
2.3
1.8


In [387]:
class Camion(object):
    
    def __init__(self, ix, carga_max, pedidos_max, dist_max):
        self.ix = ix
        self.carga_max = carga_max
        self.pedidos_max = pedidos_max
        self.dist_max = dist_max
        self.pedidos_asignados = {}
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
    def __str__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def __repr__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def _check_pedido_carga(self, pedido):
        return (self.carga_total + pedido.carga) <= self.carga_max
    
    def _check_pedido_cantidad(self, pedido):
        return self.cantidad_pedidos < self.pedidos_max
    
    def _check_pedido_distancia(self, pedido):
        dist_check = [pedido.distancia(other) <= self.dist_max for other in self.pedidos_asignados.values()]
        return all(dist_check)
    
    def check_pedido(self, pedido):
        return not pedido.asignado and self._check_pedido_carga(pedido) and self._check_pedido_cantidad(pedido) and self._check_pedido_distancia(pedido)
        # if pedido.asignado or self._check_pedido_carga(pedido) or self._check_pedido_cantidad(pedido):
        # # Falta distancias.
        #     return False
        # else:
        #     return True
    
    def add_pedido(self, pedido):
        if self.check_pedido(pedido):
            self.pedidos_asignados[pedido.ix] = pedido
            self.carga_total += pedido.carga
            self.cantidad_pedidos += 1
            pedido.asignado = True
            # Actualizar carga_total
            # Actualizar n_pedidos
    
    def remove_pedido(self, pedido_ix):
        self.pedidos_asignados.get(pedido_ix).asignado = False
        self.carga_total -= self.pedidos_asignados.get(pedido_ix).carga
        self.cantidad_pedidos -= 1
        self.pedidos_asignados.pop(pedido_ix)
        
    #def _set_carga_total(self):
    #    cargas = [ped.carga for ped in self.pedidos_asignados.values()]
    #    # Tendría que guardar en un atributo este dato?
    #    # Se tendría que actualizar cada vez que cargo un nuevo pedido?
    #    self.carga_total = sum(cargas)
    #    #return sum(cargas)
    
    def get_carga_total(self):
        return self.carga_total
    
    def get_costo(self):
        # Tendría que guardar en un atributo este dato?
        # Se tendría que actualizar cada vez que cargo un nuevo pedido?
         
        if self.carga_total == 0:
            costo = 5000
        elif self.carga_total <= 4:
            costo = 5600
        elif self.carga_total > 4 and self.carga_total < 6.5:
            costo = 1400*self.carga_total
        elif self.carga_total >= 6.5 and self.carga_total < 9.5:
            costo = 1200*self.carga_total
        else:
            costo = 1000*self.carga_total
                
        return costo 
    
    def get_costo_tn(self):
        if self.carga_total != 0:
            return self.get_costo()/self.carga_total
        else:
            return None
    
    def count_pedidos(self):
        return self.cantidad_pedidos
    
    def reset_pedidos(self):
        self.pedidos_asignados.clear()
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
        
        
        
class Pedido(object):
    
    #def __init__(self, ix):
    def __init__(self, ix, x, y, carga):
        self.ix = ix
        self.x = x
        self.y = y
        self.carga = carga
        self.asignado = False
        
    def __str__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}'
    
    def __repr__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}'
    
    def get_ix(self):
        return self.ix
    
    def get_carga(self):
        return self.carga
    
    def distancia(self, other):
        dist = ((self.x - other.x)**2 + (self.y - other.y)**2)**(1/2)
        return round(dist, 1)
    
    
    
class Ruteo(object):
    
    def __init__(self, lista_camiones, lista_pedidos, random_state):
        self.camiones = lista_camiones
        self.pedidos = lista_pedidos
        self.random_state = random_state
        
    def get_solucion_inicial(self):
        np.random.seed(self.random_state)
        for camion in self.camiones:
            for pedido in self.pedidos:
                camion.add_pedido(pedido)
            
        #camion_index = np.random.randint(0, len(self.camiones))
        #pedido_index = np.random.randint(0, len(self.pedidos))
        #self.camiones
        
    
    # PARA CAMBIAR TODA ESTA PARTE
    # GENERAR UN DATAFRAME CON TODOS LOS DATOS.
    def set_results(self):
        self.set_carga_total()
        self.set_costo_camiones()
        self.set_costo_oportunidad()
        self.set_costo_total()
        
    def get_results(self):
        self.set_results()
        print("--Resultados--")
        print(f"Carga Total {self.carga_total} tn")
        print(f"Costo Camiones {self.costo_camiones} $")
        print(f"Costo Oportunidad {self.costo_oport} $")
        print(f"Costo Total {self.costo_total} $")
    
    def set_carga_total(self):
        self.carga_total = sum([camion.get_carga_total() for camion in self.camiones])
        
    def set_costo_camiones(self):
        self.costo_camiones = sum([camion.get_costo() for camion in self.camiones])
        
    def set_costo_oportunidad(self):
        carga_no_asignada = sum([pedido.get_carga() for pedido in self.pedidos if not pedido.asignado])
        self.costo_oport = carga_no_asignada*3000
        
    def set_costo_total(self):
        self.costo_total = self.costo_camiones + self.costo_oport
        

In [300]:
[row.camion for _, row in df_camiones.iterrows()]

[1, 2, 3, 4, 5, 6]

In [305]:
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [388]:
lista_camiones = [Camion(ix=row.camion, carga_max=row.carga_max, pedidos_max=row.clientes_max, dist_max=row.dist_max) for _, row in df_camiones.iterrows()]
lista_pedidos = [Pedido(ix=row.cliente, x=row.coord_x, y=row.coord_y, carga=row.pedidos5) for _, row in df_pedidos.iterrows() if row.pedidos5 != 0]

ruteo = Ruteo(lista_camiones, lista_pedidos, 42)

In [389]:
ruteo.get_solucion_inicial()

In [391]:
for camion in ruteo.camiones:
    print(camion)
    print("------------")
    
for pedido in ruteo.pedidos:
    print(pedido)
    print("------------")
    
print("-------------")
ruteo.get_results()

Camión 1
Carga Total 9 tn
Pedidos ['A', 'B', 'E']
Costo Total 10800
Costo por tn 1200.0
------------
Camión 2
Carga Total 12 tn
Pedidos ['C', 'F']
Costo Total 12000
Costo por tn 1000.0
------------
Camión 3
Carga Total 12 tn
Pedidos ['D', 'H']
Costo Total 12000
Costo por tn 1000.0
------------
Camión 4
Carga Total 7 tn
Pedidos ['G', 'I', 'J']
Costo Total 8400
Costo por tn 1200.0
------------
Camión 5
Carga Total 10 tn
Pedidos ['K', 'L', 'N']
Costo Total 10000
Costo por tn 1000.0
------------
Camión 6
Carga Total 8 tn
Pedidos ['M']
Costo Total 9600
Costo por tn 1200.0
------------
Pedido A
Carga 6 tn
Asignado True
------------
Pedido B
Carga 1 tn
Asignado True
------------
Pedido C
Carga 6 tn
Asignado True
------------
Pedido D
Carga 8 tn
Asignado True
------------
Pedido E
Carga 2 tn
Asignado True
------------
Pedido F
Carga 6 tn
Asignado True
------------
Pedido G
Carga 4 tn
Asignado True
------------
Pedido H
Carga 4 tn
Asignado True
------------
Pedido I
Carga 2 tn
Asignado True
---

Pedido A
Carga 6 tn
Asignado True
------------
Pedido B
Carga 1 tn
Asignado True
------------
Pedido C
Carga 6 tn
Asignado True
------------
Pedido D
Carga 8 tn
Asignado True
------------
Pedido E
Carga 2 tn
Asignado True
------------
Pedido F
Carga 6 tn
Asignado True
------------
Pedido G
Carga 4 tn
Asignado True
------------
Pedido H
Carga 4 tn
Asignado True
------------
Pedido I
Carga 2 tn
Asignado True
------------
Pedido J
Carga 1 tn
Asignado True
------------
Pedido K
Carga 3 tn
Asignado True
------------
Pedido L
Carga 2 tn
Asignado True
------------
Pedido M
Carga 8 tn
Asignado True
------------
Pedido N
Carga 5 tn
Asignado True
------------
Pedido O
Carga 8 tn
Asignado False
------------


In [278]:
x = [1,2,3]
np.random.randint(0, len(x))

1

In [246]:
pedido = Pedido(ix="C", x=2, y=2, carga=11)

[pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()]

[False, False]

In [247]:
any([pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()])

False

In [250]:
not False and False

False

In [221]:
camion1.pedidos_asignados

{}

In [53]:
x = {}
x["A"] = Pedido("A")
x["B"] = Pedido("B")
print(x)
x.pop("B")
print(x)
x.clear()
print(x)

{'A': Pedido A, 'B': Pedido B}
{'A': Pedido A}
{}


In [238]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)
camion2 = Camion(ix=2, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido(Pedido(ix="A", x=1, y=1, carga=4))
camion1.add_pedido(Pedido(ix="B", x=3, y=3, carga=5))
camion2.add_pedido(Pedido(ix="C", x=1, y=1, carga=11))

print(camion1.count_pedidos())
print(camion2.count_pedidos())
print(camion1.get_carga_total())
print(camion2.get_carga_total())
print(camion1.get_costo())
print(camion2.get_costo())
print(camion1.get_costo_tn())
print(camion2.get_costo_tn())


1
1
5
11
7000
11000
1400.0
1000.0


In [263]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido(Pedido(ix="A", x=1, y=1, carga=4))
print(camion1.get_carga_total())
camion1.add_pedido(Pedido(ix="B", x=1, y=1, carga=5))
print(camion1.get_carga_total())
camion1.add_pedido(Pedido(ix="C", x=1, y=1, carga=4))
print(camion1.get_carga_total())

camion1.pedidos_asignados

True
True
True
True
True
4
True
True
True
True
True
9
True
False
True
True
False
9


{'A': Pedido A, 'B': Pedido B}

In [261]:
x = 1
x -= 1
print(x)

0


## Pruebas con datos reales

In [279]:
camion_prueba = df_camiones.iloc[0,:]
camion_prueba

camion           1
carga_max       12
clientes_max     3
dist_max         2
Name: 0, dtype: int64

In [280]:
camion = Camion(ix=camion_prueba.camion,
                carga_max=camion_prueba.carga_max,
                pedidos_max=camion_prueba.clientes_max,
                dist_max=camion_prueba.dist_max)

In [281]:
pedido_prueba = df_pedidos.iloc[0,0:2]
pedido_prueba

cliente_prueba = df_clientes.iloc[0]
cliente_prueba

Coord x    0.257571
Coord y    1.803726
Name: A, dtype: float64

In [282]:
pedido = Pedido(ix=pedido_prueba.Cliente,
                x=cliente_prueba["Coord x"],
                y=cliente_prueba["Coord y"],
                carga=pedido_prueba["Pedidos #1"])

In [283]:
camion.add_pedido(pedido)

True
True
True
True
True


Para controlar cómo copiar y guardar elementos en los camiones. Pensar qué pasaría si cargo muchos objetos en memoria.

In [284]:
# Pruebas de memoria para realizar cambios -> Ojo con modificar objetos.
camion1 = Camion("1", 2, 2, 2)
#camion2 = copy.copy(camion1)
camion2 = copy.deepcopy(camion1)
pedido1 = Pedido("A", 1, 1, 4)

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

#pedido.carga = 5
#camion.pedidos_asignados.get("A").carga = 5
camion2.pedidos_asignados.get("A").carga = 5

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

AttributeError: 'NoneType' object has no attribute 'carga'

In [175]:
class A(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class B(A):
    
    def __init__(self):
        self.x = A.x

In [176]:
a = A(1,2)
b = B(a)

TypeError: __init__() takes 1 positional argument but 2 were given